# Errors, or bugs, in your software

Today we'll cover dealing with errors in your Python code, an important aspect of writing software.

#### What is a software bug?

According to [Wikipedia](https://en.wikipedia.org/wiki/Software_bug) (accessed 16 Oct 2018), a software bug is an error, flaw, failure, or fault in a computer program or system that causes it to produce an incorrect or unexpected result, or behave in unintended ways.

#### Where did the terminology come from?

Engineers have used the term well before electronic computers and software. Sometimes Thomas Edison is credited with the first recorded use of bug in that fashion. [[Wikipedia](https://en.wikipedia.org/wiki/Software_bug#Etymology)]

#### If incorrect code is never executed, is it a bug?

This is the software equivalent to "If a tree falls and no one hears it, does it make a sound?". 

## Three classes of bugs

Let's discuss three major types of bugs in your code, from easiest to most difficult to diagnose:

1. **Syntax errors:** Errors where the code is not written in a valid way. (Generally easiest to fix.)
1. **Runtime errors:** Errors where code is syntactically valid, but fails to execute. Often throwing exceptions here. (Sometimes easy to fix, harder when in other's code.)
1. **Semantic errors:** Errors where code is syntactically valid, but contain errors in logic. (Can be difficult to fix.)

In [1]:
import numpy as np

### Syntax errors

In [6]:
print ("This should only work in Python 2.x, not 3.x used in this class.")

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("This should only work in Python 2.x, not 3.x used in this class.")? (<ipython-input-6-6a470894d2a3>, line 1)

In [7]:
x = 1; y = 2
b = x == y # Boolean variable that is true when x & y have the same value
b = 1 = 2

SyntaxError: can't assign to literal (<ipython-input-7-03308e24f762>, line 3)

### Runtime errors

In [ ]:
# invalid operation
try:
    a = 0
    5/a  # Division by zero

In [ ]:
# invalid operation
input = '40'
input/11  # Incompatiable types for the operation

### Semantic errors

Say we're trying to confirm that a trigonometric identity holds. Let's use the basic relationship between sine and cosine, given by the Pythagorean identity"

$$
\sin^2 \theta + \cos^2 \theta = 1
$$

We can write a function to check this:

In [11]:
import math

'''Checks that Pythagorean identity holds for one input, theta'''
def check_pythagorean_identity(theta):
    return math.sin(theta)**2 + math.cos(theta)**2 == 1

In [12]:
check_pythagorean_identity(.4)

True

## How to find and resolve bugs?

Debugging has the following steps:

1. **Detection** of an exception or invalid results. 
2. **Isolation** of where the program causes the error. This is often the most difficult step.
3. **Resolution** of how to change the code to eliminate the error. Mostly, it's not too bad, but sometimes this can cause major revisions in codes.


### Detection of Bugs

The detection of bugs is too often done by chance. While running your Python code, you encounter unexpected functionality, exceptions, or syntax errors. While we'll focus on this in today's lecture, you should never leave this up to chance in the future.

Software testing practices allow for thoughtful detection of bugs in software. We'll discuss more in the lecture on testing.

### Isolation of Bugs

There are three main methods commonly used for bug isolation:
1. The "thought" method. Think about how your code is structured and so what part of your could would most likely lead to the exception or invalid result.
2. Inserting ``print`` statements (or other logging techniques)
3. Using a line-by-line debugger like ``pdb``.

Typically, all three are used in combination, often repeatedly.

### Using `print` statements

Say we're trying to compute the **entropy** of a set of probabilities.  The
form of the equation is

$$
H = -\sum_i p_i \log(p_i)
$$

We can write the function like this:

In [28]:
def entropy(p):
    """
     arg p: list of float
    """
    check1 = []
    for ele in p:
        check1.append((ele <= 1) and (ele >= 0))
    check2 = 1 == sum(p)
    if all(check1) and check2:
        items = p * np.log(p)
        return -np.sum(items)
    else:
        return -1

In [16]:
p = [1.0]
items = p * np.log(p)
print(items)

[0.]


In [18]:
-sum(items)

-0.0

In [26]:
print(entropy([0.5, 0.5]))
print(entropy([0.25, 0.25, 0.25, 0.25]))

0.6931471805599453
1.3862943611198906


In [30]:
entropy([1.0, 0])

/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.


nan

If we can't easily see the bug here, let's add print statements to see the variables change over time.

## INSTRUCTOR NOTE:

1. Add print statements in tiered way, starting with simple print statements.
1. Point out that may need slight refactor on result.

    ```python
    def entropy(p):
        print(p)
        items = p * np.log(p)
        print(items)
        result = -np.sum(items)
        print(result)
        return result
    ```

1. Show complication of reading multiple print statements without labels.
1. Add labels so code looks like below.

    ```python
    def entropy(p):
        print("p=%s" % p)
        items = p * np.log(p)
        print("items=%s" % items)
        result = -np.sum(items)
        print("result=%s" % result)
        return result
    ```

In [ ]:
np.add?

Note that the print statements significantly reduce legibility of the code. We would like to remove them when we're done debugging.

In [ ]:
def entropy(p):
    items = p * np.log(p)
    return -np.sum(items)

In [ ]:
p = [0.1, 0.3, 0.5, 0.7, 0.9]
entropy(p)

Now it works fine for the first set of inputs. Let's try other inputs.

We should have documented the inputs to the function!

In [ ]:
# Create a vector of probabilities.
p = np.arange(start=5., stop=-1., step=-0.5)
p /= np.sum(p)
p

In [ ]:
entropy(p)

We get ``nan``, which stands for "Not a Number". What's going on here?

Let's add our print statements again, but it only fails later in the range of numbers. We may choose to print only if we find a `nan`.

In [ ]:
def entropy1(p):
    print("p=%s" % str(p))
    items = p * np.log(p)
    if [np.isnan(el) for el in items]:
        print(items)
    return -np.sum(items)

In [ ]:
entropy1([.1, .2])

In [ ]:
entropy1(p)

By printing some of the intermediate items, we see the problem: 0 * np.log(0) is resulting in a NaN. Though mathematically it's true that limx→0[xlog(x)]=0limx→0[xlog⁡(x)]=0, the fact that we're performing the computation numerically means that we don't obtain this result.

Often, inserting a few print statements can be enough to figure out what's going on.


In [ ]:
def entropy2(p):
    p = np.asarray(p)  # convert p to array if necessary
    print(p)
    items = []
    for val in p:
        item = val * np.log(val)
        if np.isnan(item):
            print("%f makes a nan" % val)
        items.append(item)
    #items = p * np.log(ps)
    return -np.sum(items)

In [ ]:
entropy2(p)

### Using Python's debugger, `pdb`

Python comes with a built-in debugger called [pdb](http://docs.python.org/2/library/pdb.html).  It allows you to step line-by-line through a computation and examine what's happening at each step.  Note that this should probably be your last resort in tracing down a bug.  I've probably used it a dozen times or so in five years of coding.  But it can be a useful tool to have in your toolbelt.

You can use the debugger by inserting the line
``` python
import pdb; pdb.set_trace()
```
within your script. To leave the debugger, type "exit()". To see the commands you can use, type "help".

Let's try this out:

In [33]:
def entropy(p):
    items = p * np.log(p)
    import pdb; pdb.set_trace()
    return -np.sum(items)

This can be a more convenient way to debug programs and step through the actual execution.

In [34]:
p = [.1, -.2, .3]
entropy(p)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


> <ipython-input-33-fea7073da142>(4)entropy()
-> return -np.sum(items)
(Pdb) print(p)
[0.1, -0.2, 0.3]
(Pdb) items
array([-0.23025851,         nan, -0.36119184])
(Pdb) help

Documented commands (type help <topic>):
EOF    c          d        h         list      q        rv       undisplay
a      cl         debug    help      ll        quit     s        unt      
alias  clear      disable  ignore    longlist  r        source   until    
args   commands   display  interact  n         restart  step     up       
b      condition  down     j         next      return   tbreak   w        
break  cont       enable   jump      p         retval   u        whatis   
bt     continue   exit     l         pp        run      unalias  where    

Miscellaneous help topics:
exec  pdb

(Pdb) n
--Return--
> <ipython-input-33-fea7073da142>(4)entropy()->nan
-> return -np.sum(items)
(Pdb) exit()


BdbQuit: 

In [ ]:
p = "[0.1, 0.3, 0.5, 0.7, 0.9]"
entropy(p)